# 📓 The GenAI Revolution Cookbook

**Title:** Top 7 AI Image Generation Tools with DALL·E 2 [2025 Edition]

**Description:** Unlock the full potential of AI image generation with DALL·E 2. Master API integration, prompt engineering, and output management for stunning results.

---

*This jupyter notebook contains executable code examples. Run the cells below to try out the code yourself!*



## Introduction

DALL·E 2 is OpenAI's image generation model that creates high-quality images from natural language descriptions. For developers building GenAI applications, DALL·E 2 offers a powerful API to integrate visual content generation into products—from marketing tools and design assistants to content creation platforms.

This guide walks you through the complete process of working with DALL·E 2: from API setup and authentication to generating images, handling variations, and implementing production-ready patterns. You'll learn how to craft effective prompts, manage API responses, and build reliable image generation workflows.

## Why Use DALL·E 2?

DALL·E 2 provides several advantages for developers building visual AI applications:

- **High-quality output**: Generates photorealistic images and artwork with strong coherence between text and visual elements
- **Simple API**: RESTful interface that integrates easily into existing Python applications
- **Flexible generation**: Supports image creation from text, variations of existing images, and inpainting for edits
- **Production-ready**: Built by OpenAI with enterprise-grade reliability, rate limiting, and content moderation

DALL·E 2 is ideal when you need to generate custom visual content programmatically, whether for user-facing applications, internal tools, or automated content pipelines.

## Core Concepts

Understanding these key concepts will help you work effectively with DALL·E 2:

**Prompts**: Natural language descriptions that guide image generation. Effective prompts are specific, descriptive, and include details about style, composition, and subject matter.

**Image Generation**: The primary operation that creates images from text prompts. You specify size (256x256, 512x512, or 1024x1024) and quantity (1-10 images per request).

**Variations**: Generate alternative versions of an existing image while maintaining its core characteristics. Useful for exploring design options or creating similar content.

**Edits (Inpainting)**: Modify specific regions of an image using a mask and text prompt. This allows targeted changes without regenerating the entire image.

**Response Format**: Images are returned as URLs (default) or base64-encoded data, depending on your application's needs.

## Setup

Install the OpenAI Python client:

In [ ]:
!pip install openai pillow requests

Import required libraries and configure authentication:

In [ ]:
import openai
import os
from PIL import Image
import requests
from io import BytesIO

# Set your OpenAI API key
openai.api_key = os.getenv("OPENAI_API_KEY")  # or set directly: "your-api-key-here"

Obtain your API key from the [OpenAI platform](https://platform.openai.com/api-keys). Store it securely using environment variables rather than hardcoding it in your scripts.

## Generating Images from Text

The core functionality of DALL·E 2 is creating images from text descriptions:

In [ ]:
def generate_image(prompt, size="1024x1024", n=1):
    """
    Generate images using DALL·E 2
    
    Args:
        prompt: Text description of the desired image
        size: Image dimensions (256x256, 512x512, or 1024x1024)
        n: Number of images to generate (1-10)
    """
    response = openai.Image.create(
        prompt=prompt,
        n=n,
        size=size
    )
    return response['data']

# Generate a single image
prompt = "A futuristic city skyline at sunset with flying cars, digital art style"
images = generate_image(prompt, size="1024x1024", n=1)

# The response contains URLs to the generated images
image_url = images[0]['url']
print(f"Generated image URL: {image_url}")

Download and display the generated image:

In [ ]:
def download_image(url):
    """Download image from URL and return PIL Image object"""
    response = requests.get(url)
    return Image.open(BytesIO(response.content))

# Download and save the image
img = download_image(image_url)
img.save("generated_image.png")
print("Image saved as generated_image.png")

**Prompt Engineering Tips**:
- Be specific about style, mood, and composition
- Include details about lighting, perspective, and medium
- Use descriptive adjectives and clear subject definitions
- Experiment with artistic styles (e.g., "oil painting", "3D render", "watercolor")

## Creating Image Variations

Generate variations of an existing image to explore alternative designs:

In [ ]:
def create_variation(image_path, n=1, size="1024x1024"):
    """
    Create variations of an existing image
    
    Args:
        image_path: Path to the source image (must be PNG, < 4MB, square)
        n: Number of variations to generate
        size: Output image size
    """
    with open(image_path, "rb") as image_file:
        response = openai.Image.create_variation(
            image=image_file,
            n=n,
            size=size
        )
    return response['data']

# Create variations of the generated image
variations = create_variation("generated_image.png", n=2)

# Download variations
for i, var in enumerate(variations):
    img = download_image(var['url'])
    img.save(f"variation_{i}.png")
    print(f"Saved variation_{i}.png")

**Important**: The source image must be a square PNG file under 4MB. Resize and convert images if needed before creating variations.

## Editing Images with Inpainting

Use inpainting to modify specific regions of an image:

In [ ]:
def edit_image(image_path, mask_path, prompt, n=1, size="1024x1024"):
    """
    Edit an image using a mask and text prompt
    
    Args:
        image_path: Path to the original image (PNG, < 4MB, square)
        mask_path: Path to the mask image (PNG, transparent areas indicate edit regions)
        prompt: Description of the desired edit
        n: Number of edited versions to generate
        size: Output image size
    """
    with open(image_path, "rb") as image_file, open(mask_path, "rb") as mask_file:
        response = openai.Image.create_edit(
            image=image_file,
            mask=mask_file,
            prompt=prompt,
            n=n,
            size=size
        )
    return response['data']

# Example: Edit a specific region
# You need to create a mask image where transparent areas indicate what to edit
edited = edit_image(
    "generated_image.png",
    "mask.png",
    "Add a bright full moon in the sky",
    n=1
)

img = download_image(edited[0]['url'])
img.save("edited_image.png")

Create masks programmatically using PIL:

In [ ]:
from PIL import Image, ImageDraw

def create_circular_mask(image_path, center, radius, output_path):
    """Create a circular mask for inpainting"""
    img = Image.open(image_path).convert("RGBA")
    mask = Image.new("RGBA", img.size, (0, 0, 0, 255))  # Opaque black
    draw = ImageDraw.Draw(mask)
    
    # Draw transparent circle (area to edit)
    left = center[0] - radius
    top = center[1] - radius
    right = center[0] + radius
    bottom = center[1] + radius
    draw.ellipse([left, top, right, bottom], fill=(0, 0, 0, 0))
    
    mask.save(output_path)
    return output_path

# Create a mask for the top-right region
create_circular_mask("generated_image.png", center=(768, 256), radius=200, output_path="mask.png")

## Building a Production-Ready Generator

Implement error handling, retries, and response management for production use:

In [ ]:
import time
from typing import List, Optional

class DALLEGenerator:
    """Production-ready DALL·E 2 image generator with error handling"""
    
    def __init__(self, api_key: str, max_retries: int = 3):
        openai.api_key = api_key
        self.max_retries = max_retries
    
    def generate(self, prompt: str, size: str = "1024x1024", n: int = 1) -> List[str]:
        """Generate images with retry logic"""
        for attempt in range(self.max_retries):
            try:
                response = openai.Image.create(
                    prompt=prompt,
                    n=n,
                    size=size
                )
                return [img['url'] for img in response['data']]
            
            except openai.error.RateLimitError:
                if attempt < self.max_retries - 1:
                    wait_time = 2 ** attempt  # Exponential backoff
                    print(f"Rate limit hit. Waiting {wait_time}s...")
                    time.sleep(wait_time)
                else:
                    raise
            
            except openai.error.InvalidRequestError as e:
                print(f"Invalid request: {e}")
                raise
            
            except Exception as e:
                print(f"Unexpected error: {e}")
                if attempt < self.max_retries - 1:
                    time.sleep(1)
                else:
                    raise
    
    def download_and_save(self, url: str, filepath: str) -> None:
        """Download image from URL and save to disk"""
        try:
            response = requests.get(url, timeout=30)
            response.raise_for_status()
            
            img = Image.open(BytesIO(response.content))
            img.save(filepath)
            print(f"Saved image to {filepath}")
        
        except Exception as e:
            print(f"Failed to download image: {e}")
            raise

# Usage example
generator = DALLEGenerator(api_key=os.getenv("OPENAI_API_KEY"))

prompts = [
    "A serene mountain landscape with a lake reflection, photorealistic",
    "Abstract geometric patterns in vibrant colors, modern art style",
    "A cozy coffee shop interior with warm lighting, architectural photography"
]

for i, prompt in enumerate(prompts):
    print(f"\nGenerating image {i+1}: {prompt}")
    urls = generator.generate(prompt, size="1024x1024", n=1)
    generator.download_and_save(urls[0], f"output_{i}.png")

## Run and Evaluate

Test the complete workflow:

In [ ]:
# Initialize generator
generator = DALLEGenerator(api_key=os.getenv("OPENAI_API_KEY"))

# Test prompt
test_prompt = "A robot reading a book in a library, warm lighting, digital illustration"

# Generate image
print("Generating image...")
image_urls = generator.generate(test_prompt, size="512x512", n=1)

# Download and save
generator.download_and_save(image_urls[0], "test_output.png")

# Create variations
print("\nCreating variations...")
variations = create_variation("test_output.png", n=2, size="512x512")

for i, var in enumerate(variations):
    generator.download_and_save(var['url'], f"test_variation_{i}.png")

print("\nGeneration complete. Check the output files.")

Monitor API usage and costs:

In [ ]:
# Track generation costs (approximate)
def estimate_cost(size: str, n: int) -> float:
    """Estimate cost for DALL·E 2 generation (as of 2024)"""
    costs = {
        "1024x1024": 0.020,  # $0.020 per image
        "512x512": 0.018,    # $0.018 per image
        "256x256": 0.016     # $0.016 per image
    }
    return costs.get(size, 0.020) * n

# Calculate cost for batch generation
total_cost = sum(estimate_cost("1024x1024", 1) for _ in prompts)
print(f"Estimated cost for batch: ${total_cost:.3f}")

## Conclusion

You now have a complete foundation for integrating DALL·E 2 into your applications. You've learned how to generate images from text, create variations, perform targeted edits, and implement production-ready patterns with error handling and retry logic.

**Key takeaways**:
- Effective prompt engineering significantly impacts output quality
- Always implement retry logic and error handling for API calls
- Consider image size trade-offs between quality and cost
- Use variations and edits to refine outputs iteratively

**Next steps**:
- Integrate DALL·E 2 into a web application or API service
- Combine with GPT models for automated prompt generation
- Implement caching strategies to reduce API costs
- Explore content moderation and safety filters for user-generated prompts

For complete API documentation and updates, visit the [OpenAI DALL·E API reference](https://platform.openai.com/docs/guides/images).